# Assess MDPO Results

Loads MDPO optimization pickle files and evaluates training dynamics, convergence, and resultant device trajectories.
Also equipped to perform SEPIO classification assessments on device placements to estimate achievable information throughput and classification accuracy limits.

Overview:
- MDPO (Multi-Device Placement Optimization) produces candidate device trajectories via genetic / annealing / gradient strategies.
- This notebook ingests one or more MDPO result pickle files, extracts best solutions per generation, and summarizes fitness progression.
- Rebuilds sensing voltage matrices for each device set to enable downstream SEPIO Monte Carlo classification benchmarking.

Primary Inputs:
- Pickle files containing MDPO run dictionaries (see key list in loading section).
- Brain / ROI surface .mat files matching those used during optimization.
- Lead field (.npz) files defining sensor-specific field vectors.

Primary Outputs:
- Plots: fitness vs device count, fitness vs generations/temperature, accuracy vs sensor count, sensor-by-device contour.
- Arrays: voltage tensors per device set suitable for SEPIO training.
- Summaries: Information Capacity (IC) and voltage statistics, distribution analyses by distance.

Workflow Steps:
1. Configure anatomical and lead field resources (matching optimization settings).
2. Load MDPO pickle(s) and inspect key/value structure.
3. Extract and optionally convert angles (radians vs degrees) for device trajectories.
4. Reconstruct per-source voltages for each optimized device set.
5. Compute IC / SNR summaries and voltage distributions.
6. Run SEPIO Monte Carlo classification across sensor subsets.
7. Visualize accuracy scaling and device/sensor trade-offs.

Notes:
- Ensure all anatomical and lead field settings match those used during original MDPO optimization or results will be invalid.
- SEPIO training treats voltages as feature space; spatial accuracy can optionally be computed if implemented.
- Information Capacity calculation assumes global `bandwidth` and `noise` constants consistent with device modality.


In [ ]:
### Imports and Files ###
# Core scientific / IO libraries
import matplotlib.pyplot as plt   # Plotting utilities for fitness, accuracy, distributions
import numpy as np                # Numerical arrays and vectorized operations
from os import path               # Path joins (portable path construction)
import sys                        # Modify import search path for local modules
import pickle                     # Load MDPO optimization result dictionaries

# Project module path insertion (two levels up to root for `modules/`)
sys.path.insert(0, path.join('../..'))

# Local project modules
from modules.leadfield_importer import FieldImporter  # Loads lead field (.npz) arrays
from scipy.io import loadmat                          # Reads MATLAB .mat surface/ROI files
from modules.SEPIO import sepio                       # SEPIO classification / MC training routines


Provides graphing of MDPO results and performs SEPIO on desired sensor sets

## Finding device locations
Settings __MUST__ match those chosen prior in optimization!

Can potentially switch devices __IF__ they are the same shape lead fields (excluding sensor axis).

In [ ]:
# Brainsuite files / anatomical resource configuration
# Provide full brain and ROI geometry sets identical to those used during MDPO optimization.
# Each entry in brain_data / roi_data corresponds to a separate .mat surface file with
# faces, vertices, normals accessible under specified struct keys.
folder = r"...\SEPIO_dataset" # Root dataset folder (UPDATE to actual absolute path before running)

# --- Single ROI Demo (Broca's area) ---
brain_data = [loadmat(path.join(folder,'MDPO_data','brain.mat'))]  # Full brain surface
roi_data = [loadmat(path.join(folder,'MDPO_data','r_broca.mat'))]  # ROI surface subset
roi_weights = [1]              # Uniform weight (can scale relative importance across multiple ROIs)
roi_label = ['ans']            # MATLAB struct key in ROI files (e.g., 'ans','auditory_roi')
brain_label = ['brain']        # MATLAB struct key in brain files (e.g., 'brain','ans')

do_weights = False            # Toggle: apply ROI weights to voltage/SNR/IC metrics

"""
# --- Multi-Region Example (ACC + left/right DLPFC/FEF) ---
# Uncomment and adjust if evaluating multi-region optimization results.
roi_data = [loadmat(path.join(folder,'ACC_inner_cortex.mat')),            # ACC inner cortex
            loadmat(path.join(folder,'L-DLPFC_FEF.mat')),                # Left DLPFC/FEF
            loadmat(path.join(folder,'R-DLPFC_FEF.mat'))]                # Right DLPFC/FEF
brain_data = [loadmat(path.join(folder,'L_inner_cortex.mat')),           # Left hemisphere inner cortex
              loadmat(path.join(folder,'R_inner_cortex.mat'))]           # Right hemisphere inner cortex
roi_weights = [1.,0.5,0.5]        # Weighted emphasis for subregions (ACC higher priority)
roi_label = ['ans','ans','ans']   # Struct keys for ROI files
brain_label = ['ans','ans']       # Struct keys for brain files

do_weights = False               # Enable to scale metrics by roi_weights
"""

# Dipole / biophysical parameters
# Magnitude: 0.5e-9 nAm typical physiologic; 20e-9 nAm for phantom simulations
magnitude = 0.5e-9        # nAm dipole moment used for voltage synthesis
dipole_base_length = 2.5  # mm approximate cortical column length (contextual reference)

# Lead field selection (ensure match with MDPO run; sensor geometry must be identical)
fields_files = path.join(folder,'leadfields',"SEEG_8e_500um_1500pitch.npz")  # SEEG example
#fields_files = path.join(folder,'leadfields',"DISC_30mm_p2-5Sm_MacChr.npz") # DiSc example
#fields_files = path.join(folder,'leadfields',"ECoG_1mm_500umgrid.npz")       # ECoG example

# DiSc specific aggregation parameters (ignored for SEEG/ECoG)
macros = False          # If True, reduce DiSc electrodes into virtual macro rings
virtual_rings = 4       # Number of macro rings (must be one of {1,2,4,8})

# Processing / sensing environment parameters
sensor_div = 8          # Sensor granularity for SEPIO sweeps (must divide sensor_max evenly)
                        # Recommended: 16 for DiSc; 2 or 4 for SEEG depending on electrode count
scale = 0.5             # mm per voxel (lead field spatial resolution)
cl_wd = 1.0             # mm clearance diameter (legacy; may be unused)
noise = 4.1             # μV RMS noise; 4.1 DiSc | 2.7 SEEG | varies for ECoG by diameter
bandwidth = 100         # Samples per second (used for information capacity)
Montage = False         # If True, compute montage differences (max-min) rather than single optimal

# Lead field loading
field_importer = FieldImporter()
field = field_importer.load(fields_files)
num_electrodes = np.shape(field_importer.fields)[4]  # Total electrodes in selected lead field
fields = field_importer.fields
midpoint = fields.shape[0]//2  # Lead field cube midpoint index (X/Y assumed square)


In [ ]:
# Genetic / Anneal / Gradient (MDPO) result file handling
# 'files' should list MDPO pickle result filenames located under folder/MDPO_data.
# Each pickle expected to contain a dictionary capturing optimization metadata and results.
# See key inventory below for structure expectations.

# GA file locations
out = path.join(folder,"outputs")  # Output directory for plots / summaries

# Auditory cortex experiment set (paper example)
files = [r"genetic-1.pkl",r"genetic-2.pkl",r"genetic-3.pkl",r"genetic-4.pkl",r"genetic-5.pkl"]

# Dual region ACC / DLPFC-FEF example set
#files = [r"ACC-1.pkl",r"ACC-gradient-2.pkl",r"ACC-gradient-3.pkl",
#         r"ACC-gradient-4.pkl",r"ACC-gradient-5.pkl"]

# Assign file N (device counts). For multi-device optimization runs, user must ensure
# N_file matches number of devices represented in each pickle (or derive automatically).
N_file = np.arange(1,len(files)+1)  # Integer device counts per file (sequential assumption)
pickle_files = []
for f in files:
    pickle_files.append(path.join(folder,'MDPO_data',f))

# Load Pickle files into `data` list
# Each element: dict with keys like 'best solution:', 'fitnesses', 'device types', etc.
data = []
for p in pickle_files:
    with open(p,'rb') as f:
        data.append(pickle.load(f))

# Assess data dict structure (verbosity intended for exploratory inspection)
''' KEYS (Representative)
'type'                      : Optimization method descriptor ('genetic','anneal','gradient')
'best solution:'            : Flat array of concatenated device poses [x,y,z,alpha,beta,gamma]*N
'best solution fitness'     : Scalar fitness value (e.g., total information capacity)
'fitnesses'                 : List of fitness values per generation / iteration
'best solution over generations' : List tracking best-so-far solution states
'mean fitnesses'            : Mean population fitness per generation (genetic)
'std fitnesses'             : Std dev of population fitness per generation
'device names' / 'device types'  : Metadata describing each device instance
'field files'               : Lead field file references ensuring reproducibility
'N per device'              : Sensor counts per device if heterogeneous
'LF scale'                  : Spatial scale used (mm per voxel)
'dev diameter' / 'dev depth': Physical device geometry parameters
'dev backend'               : Recording backend identifier (if provided)
'voltage scale'             : Scaling factor used inside optimization
'dev bandwidth'             : Effective sampling bandwidth
'depth limits' / 'angle limits' : Constraints applied during search
'montaged'                  : Boolean indicating montage-enabled fitness evaluation
'''

best = []
radians = False # Return radians or degrees; MUST be FALSE for SEPIO; MUST be TRUE for visualization plotting
if radians:
    print("!!! DISABLE `radians` to use SEPIO (expects degrees in original pickle) !!!")

print('Data Summary:')
for i,f in enumerate(files):
    print("N =",N_file[i])
    depth = sum(isinstance(v, dict) for v in data[i].values()) # counts sub-dicts
    bredth = len(next(iter(data[i].values()))) # counts dict bredth
    print('Depth:',depth,'\nBredth',bredth)
    for key, value in data[i].items():
        print('key',key)#, value) # Prints all keys and values
    best.append(data[i]['best solution:']) # Degrees
    
    # Optional conversion to degrees (if underlying storage was radians)
    if radians:
        for j in np.arange(N_file[i]): # j for each device
            best[i][6*j+3] = best[i][6*j+3]*180/np.pi
            best[i][6*j+4] = best[i][6*j+4]*180/np.pi
            best[i][6*j+5] = best[i][6*j+5]*180/np.pi
    print(f"File #{i+1}:")
    print(best[i].reshape(best[i].shape[0]//6,6))

NOTE: 'best solution' and 'best solution fitness' are swapped in early anneal and gradient methods

## Plotting fitness

In [ ]:
data = data[0]
best_solution = data['best solution:']
best_solution_fitness = data['best solution fitness']
fitnesses = data['fitnesses']
best_sols = data['best solution over generations']
mean_fitnesses = data['mean fitnesses']
std_fitnesses = data['std fitnesses']

N = np.array(data['best solution:']).shape[0]//6
num_generations = len(data['fitnesses']) # Adjusts to loaded data
generations = list(np.arange(0,num_generations,1))

for gen in generations:
    best_sol = best_sols[gen].reshape((N, 6))

In [ ]:
### Graph best fitness scores versus device count ###
fitness = np.empty((len(N_file),))
for i in range(len(N_file)): # i for each file
    fitness[i] = data[i]['best solution fitness']

plt.plot(N_file,fitness/1000.,label="Total IC")
for i,f in enumerate(fitness):
    fitness[i] = f/N_file[i]
plt.plot(N_file,fitness/1000.,label="IC per device")
#plt.yscale('log')
plt.xticks(N_file)
plt.ylabel('Information Capacity (kBits/s.)')
plt.xlabel('Number of Devices')
plt.title(f"Fitness per device\n({data[0]['type'][0]})")
plt.legend(loc='right')
plt.savefig(path.join(out,f"fitness_per_device-{data[0]['type'][0]}.pdf"),dpi=300)
plt.show()

In [ ]:
### Graph the fitness gain over generations for all device counts ###
if data[0]['type'][0] == 'anneal':
    # Make graph relative to heat, not generations
    fitness = []
    heat = []
    for i in np.arange(len(N_file)): # i for each file
        f = []
        h = []
        for k in np.arange(len(data[i]['fitnesses'])): # k for each fitness/heat per file
            f.append(data[i]['fitnesses'][k])
            h.append(data[i]['temperature'][k])
        fitness.append(f)
        heat.append(h)

    for i,n in enumerate(N_file):
        if n == 1:
            plt.plot(heat[i],fitness[i],label=f'{n} Device')
        else:
            plt.plot(heat[i],fitness[i],label=f'{n} Devices')
    #plt.yscale('log')
    plt.ylabel('Information Capacity (Bits/s.)')
    plt.xlabel('Temperature')
    plt.xscale('log')
    plt.gca().invert_xaxis()
    plt.title(f"Fitness per device over annealing temp.\n({data[0]['type'][0]})")
    plt.legend(loc='lower right')
    plt.savefig(path.join(out,f"fitness_over_generations-{data[0]['type'][0]}.pdf"),dpi=300)
    plt.show()
else:
    num_generations = len(data[0]['fitnesses'])
    generations = list(np.arange(0,num_generations,1))
    fitness = []# np.zeros((len(N_file),num_generations))
    for i in np.arange(len(N_file)): # i for each file
        fitness.append(data[i]['fitnesses'])

    for i,n in enumerate(N_file):
        if n == 1:
            plt.plot(range(len(fitness[i])),np.array(fitness[i])/1000.,label=str(n)+' Device')
        else:
            plt.plot(range(len(fitness[i])),np.array(fitness[i])/1000.,label=str(n)+' Devices')
    #plt.yscale('log')
    plt.ylabel('Information Capacity (kBits/s.)')
    plt.xlabel('Generations')
    plt.title(f"Fitness per device over generations\n({data[0]['type'][0]})")
    plt.legend(loc='lower right')
    plt.savefig(path.join(out,f"fitness_over_generations-{data[0]['type'][0]}.pdf"),dpi=300)
    plt.show()

## Re-generating voltages and performing SEPIO

### Functions

In [ ]:
def virtual_macros(field,rings):
    """Aggregate a high-density DiSc lead field into virtual macro electrodes.

    Parameters
    ----------
    field : np.ndarray
        Lead field array of shape [X, Y, Z, 3, E] where E=64 for standard DiSc.
    rings : int
        Number of macro ring divisions (must be one of {1,2,4,8}). Determines how rows
        are grouped vertically to form averaged macro contacts.

    Returns
    -------
    new_field : np.ndarray
        Reduced lead field with shape [X, Y, Z, 3, rings], each channel an averaged macro.
        If input shape unsupported, original field returned with warning.

    Notes
    -----
    - Averaging preserves overall magnitude by dividing by number of electrodes per macro.
    - Assumes 8 columns x 8 rows layout (64 electrodes).
    """
    rings = int(rings)
    if field.shape[-1] == 64: # 64 ch DiSc
        rows_per_ring = 8//rings
        e_per_ring = rows_per_ring*8
        new_field = np.zeros((field.shape[0],field.shape[1],field.shape[2],3,rings))
        for r in range(rings): # r for each ring; top down
            for c in range(8): # c for each column
                for d in range(rows_per_ring): # d for column depth in each ring
                    new_field[:,:,:,:,r] += field[:,:,:,:,c*8+r*rows_per_ring+d]
        new_field /= e_per_ring
        return new_field
    else:
        print("Incorrect input LF shape.")
        return field

def field_reduce(field,count):
    """Reduce an original high-density lead field (e.g., 128 ch DiSc) to a lower sensor count.

    Parameters
    ----------
    field : np.ndarray
        Lead field array [X,Y,Z,3,E] with E typically 128.
    count : int
        Desired electrode count subset (currently supports 128 -> 64 reduction).

    Returns
    -------
    field_out : np.ndarray | None
        Reduced field with selected electrodes, or None if invalid count requested.

    Notes
    -----
    - For 128->64, keeps every other electrode index (even slicing).
    - No interpolation performed; strict channel subsampling.
    """
    if field.shape[-1] == 128:  # Starting with 128 channels
        if count == 128:
            field_out = field
        if count == 64:
            keep = np.arange(0,128,2)
            field_out = field[:,:,:,:,keep]
        else:
            print("ERROR: No valid sensor count provided!")
            field_out = None # break process
    else:
        print("ERROR: Original LF not provided. LF unchanged.")
        field_out = field
    return field_out

def obtain_data(data, name):
    """Extract surface mesh components from a MATLAB .mat structure.

    Parameters
    ----------
    data : dict
        Result of `scipy.io.loadmat` containing region surface under key `name`.
    name : str
        Struct name containing (faces, vertices, normals) triple.

    Returns
    -------
    header : bytes
        MAT-file header metadata.
    faces : np.ndarray (F,3)
        Triangle indices.
    vertices : np.ndarray (V,3)
        Vertex coordinates.
    normals : np.ndarray (V,3)
        Per-vertex normal vectors.
    """
    header = data['__header__']
    faces = data[name][0][0][0]
    vertices = data[name][0][0][1]
    normals = data[name][0][0][2]

    return header, faces, vertices, normals


def recenter(vertices, reference):
    """Recenter a set of vertices around the center-of-mass of a reference vertex cloud.

    Parameters
    ----------
    vertices : np.ndarray (N,3)
        Vertex coordinates to translate.
    reference : np.ndarray (M,3)
        Reference coordinates whose mean defines new origin.

    Returns
    -------
    vertices_modified : np.ndarray (N,3)
        Recentered vertex array.
    """
    center = np.mean(reference, axis = 0)
    avg1 = center[0]
    avg2 = center[1]
    avg3 = center[2]
    vertices_modified = vertices.copy()

    # modify the data according to the midpoint
    for idx in range(vertices.shape[0]):
        vertices_modified[idx][0] -= avg1
        vertices_modified[idx][1] -= avg2
        vertices_modified[idx][2] -= avg3

    return vertices_modified

def get_rotmat(alpha, beta, gamma):
    """Compute combined rotation matrix from Euler angles (yaw, pitch, roll).

    Parameters
    ----------
    alpha : float
        Yaw angle in radians (rotation about Z-axis).
    beta : float
        Pitch angle in radians (rotation about Y-axis).
    gamma : float
        Roll angle in radians (rotation about X-axis).

    Returns
    -------
    rot_mat : np.ndarray (3,3)
        Rotation matrix R = yaw * pitch * roll.
    """
    yaw = np.array([[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]])
    pitch = np.array([[np.cos(beta), 0, np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]])
    roll = np.array([[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]])
    rot_mat = np.matmul(yaw, pitch)
    rot_mat = np.matmul(rot_mat, roll)

    return rot_mat

def transform_vectorspace(vertices, normals, devpos, max_depth = None):
    """Transform cortical vertices/normals into device-local coordinate space.

    Parameters
    ----------
    vertices : np.ndarray (N,3)
        Recentered cortical layer vertices (mm).
    normals : np.ndarray (N,3)
        Corresponding normal vectors.
    devpos : array-like (6,)
        Device pose [x,y,z,alpha,beta,gamma] (mm, radians).
    max_depth : float | None
        Optional depth constraint (placeholder; not currently enforced).

    Returns
    -------
    dippos : np.ndarray (N,3)
        Transformed dipole positions in voxel space (int-cast floats).
    dipvec : np.ndarray (N,3)
        Dipole vectors scaled by global `magnitude`.
    rot_mat : np.ndarray (3,3)
        Forward rotation matrix used (inverse applied during transformation).

    Notes
    -----
    - Applies inverse rotation to align dipoles relative to device orientation.
    - X/Y shifted by half field size to center within lead field volume.
    - Unit conversion mm -> voxel index via global `scale`.
    """
    rot_mat = get_rotmat(devpos[3],devpos[4],devpos[5])
    inv_rot_mat = np.linalg.inv(rot_mat)
    dippos = np.copy(vertices)
    dipvec = np.copy(normals)

    # translation
    dippos-=devpos[:3]
    dippos[0] += fields.shape[0]//2
    dippos[1] += fields.shape[1]//2

    # rotate vertices and normals based on the placement of the device
    for idx in range(vertices.shape[0]):   # rotate for each vertex point and corresponding vectors
        dippos[idx] = np.matmul(inv_rot_mat, dippos[idx])
        dipvec[idx] = np.matmul(inv_rot_mat, dipvec[idx])

    # Transfer position into leadfield space
    dippos *= 1/scale

    # to make it compatible with the lead field data
    dippos = (dippos.astype('int')).astype('float')

    # check if the depth restriction is exceeded
    if max_depth is not None:
        if dippos[2].any() < max_depth:
            # Placeholder for depth enforcement logic
            pass
    
    # Scale to dipole nAm magnitude, see value above
    dipvec *= magnitude 

    return dippos, dipvec, rot_mat

def trim_data(leadfield, vertices, normals):
    """Mask dipole positions lying outside the lead field bounds.

    Parameters
    ----------
    leadfield : np.ndarray
        Lead field array [X,Y,Z,3,E].
    vertices : np.ndarray (N,3)
        Candidate dipole positions (voxel indices).
    normals : np.ndarray (N,3)
        Dipole moment vectors (scaled normals).

    Returns
    -------
    dippos : np.ndarray (N,3)
        Positions with out-of-volume entries replaced by NaNs.
    dipvec : np.ndarray (N,3)
        Unchanged dipole vectors.

    Notes
    -----
    - Z dimension assumed doubled relative to X/Y extents (checked explicitly).
    - NaNs propagate through later metric calculations to exclude invalid points.
    """
    dippos = np.copy(vertices)
    dipvec = np.copy(normals)
    # half of side length
    len_half = leadfield.shape[0]//2

    # um = 0
    for idx in range(vertices.shape[0]):
        if((np.abs(vertices[idx][0])>len_half) or (np.abs(vertices[idx][1])>len_half) or (vertices[idx][2])>len_half*2) or (vertices[idx][2]<0):
            dippos[idx] = np.nan

    return dippos, dipvec

def calculate_voltage(fields, vertices, normals, vscale=10**6, montage = Montage, inter = False):
    """Compute voltage, SNR, and information capacity per dipole location.

    Parameters
    ----------
    fields : np.ndarray
        Lead field array [X,Y,Z,3,E].
    vertices : np.ndarray (N,3)
        Transformed dipole positions (NaNs for invalid).
    normals : np.ndarray (N,3)
        Dipole vectors (nAm) oriented per device.
    vscale : float
        Scaling from Volts to microvolts (default 1e6).
    montage : bool
        If True use max-min montage spread where beneficial.
    inter : bool
        Placeholder for future inter-device montage support.

    Returns
    -------
    opt_volt : np.ndarray (N,)
        Optimal voltage per dipole (μV).
    snr_list : np.ndarray (N,)
        SNR values (dimensionless power ratio).
    info_cap : np.ndarray (N,)
        Information capacity estimates (bits/s.).
    voltage : np.ndarray (N,E)
        Raw electrode voltages before selection (μV after scaling).

    Notes
    -----
    - SNR = (Voltage / noise)^2; noise defined globally.
    - Information Capacity = bandwidth * log2(1 + SNR).
    - Optional ROI weighting applies multiplicative scaling if enabled.
    """
    global roi_vertices_weights, do_weights
    
    # make copies of data
    dippos = np.copy(vertices)
    dipvec = np.copy(normals)

    # create a list of field vectors applicable to the surface, [vertex index, electrode #]
    voltage = np.empty((dippos.shape[0], fields.shape[-1]))
    voltage[:]=np.nan
    opt_volt = np.empty(dippos.shape[0])
    opt_volt[:]=np.nan
    
    # calculate voltage for each vertex
    for idx in range(dippos.shape[0]): # for each dipole
        for e in range(fields.shape[-1]): # for every electrode
            if not np.any(np.isnan(dippos[idx])): # if not nan, calculate and modify entry
                try:
                    lead_field = fields[int(dippos[idx][0]+fields.shape[0]//2), int(dippos[idx][1]+fields.shape[0]//2), int(dippos[idx][2]), :, e]
                    voltage[idx, e] = np.dot(lead_field, dipvec[idx])
                except: # error in finding lead field
                    voltage[idx, e] = np.nan
        if (not np.all(np.isnan(voltage[idx,:]))) and (not montage): 
            opt_volt[idx] = np.nanmax(np.abs(np.copy(voltage[idx]))) # get the optimal voltage across all electrodes
        elif (not np.all(np.isnan(voltage[idx,:]))) and montage and (not inter):
            single = np.nanmax(np.abs(np.copy(voltage[idx]))) # get the optimal voltage across all electrodes
            mont = np.nanmax(voltage[idx]) - np.nanmin(voltage[idx])
            if mont > single: # Use montage if it's better
                opt_volt[idx] = mont
            else: # Keep zero reference if it's better
                opt_volt[idx] = single
        # elif (not np.all(np.isnan(voltage[idx,:]))) and montage and inter:
    voltage *= vscale # (sources, electrodes) for one device
    opt_volt *= vscale #scale it to get the list of voltages
    snr_list = np.square(np.copy(opt_volt)/noise) # get the list of snr values
    info_cap = bandwidth*np.log2(1+snr_list) # get the list of information capacity
    snr_list[np.isnan(snr_list)] = 0

        # Scale with given weights for ROI subregions
    if do_weights:
        opt_volt = np.multiply(opt_volt,roi_vertices_weights)
        snr_list = np.multiply(snr_list,roi_vertices_weights)
        info_cap = np.multiply(info_cap,roi_vertices_weights)


    return (opt_volt, snr_list, info_cap, voltage)

def find_snr(devpos,value):
    """Aggregate sensing metrics across multiple devices for a given pose list.

    Parameters
    ----------
    devpos : np.ndarray (N*6,)
        Flat array of concatenated device poses [x,y,z,alpha,beta,gamma] * N devices.
    value : int
        Metric selector: 0 voltage(montaged), 1 SNR, 2 IC, 3 or -1 raw voltage tensor.

    Returns
    -------
    all_dev_volt : np.ndarray
        Metric array with shape depending on selection. For IC: (sources, N). For raw voltage:
        (sources, N, electrodes). For others: (sources, N, electrodes) pre-aggregation.

    Notes
    -----
    - Uses global `fields`, `roi_normals`, `recentered_roi`, `Montage`.
    - Result may contain NaNs which are later converted or aggregated.
    """
    devpos = devpos.reshape((N, 6))
    if value == 2: # IC
        all_dev_volt = np.empty((recentered_roi.shape[0], N))
    else:
        all_dev_volt = np.empty((recentered_roi.shape[0], N, fields.shape[-1]))
    for idx in range(len(devpos)):
        dippos, dipvec, rotmat = transform_vectorspace(recentered_roi, roi_normals, devpos[idx])
        dippos_adj, dipvec_adj = trim_data(fields, dippos, dipvec)
        # Index decides volt, snr, IC, or voltage by(source,electrodes)
        if value == 2: # IC
            all_dev_volt[:, idx] = calculate_voltage(fields, dippos_adj, dipvec_adj, montage = Montage, inter = False)[value]
        else:
            all_dev_volt[:, idx,:] = calculate_voltage(fields, dippos_adj, dipvec_adj, montage = Montage, inter = False)[value]
    
    voltage = np.nanmax(np.nan_to_num(all_dev_volt, nan=0), axis=1)
    total = np.nansum(voltage)
    # print("total IC:", total, "Bits/s")
    return all_dev_volt

### Voltage calculations

In [ ]:
### Adjust fields to desired size ###
disc_sensors = 64 # 64 or 128 for DiSc; sensors reduced evenly  in grid
fields = field_reduce(fields,disc_sensors)

In [ ]:
### Calculate virtual macros from DiSc ###
if macros:
    fields = virtual_macros(fields,virtual_rings)

In [ ]:
### Load brain and ROI ###
# Values to collect
brain_vals = []
brain_vallen = [[],[],[]] # summative index for each start point
brain_count = [0,0,0] # running index sum
roi_vals = []
roi_vallen = [[],[],[]]
roi_count = [0,0,0]
# Start indexing counts per file
#for b in brain_data:
#    brain_count.append(0)
#for b in roi_data:
#    roi_count.append(0)

# Load each file
for i,fb in enumerate(brain_data):
    _, brain_faces, brain_vertices, brain_normals = obtain_data(brain_data[i], brain_label[i]) # 'ans' or 'brain'
    brain_vals.append([brain_faces,brain_vertices,brain_normals])
    for k in range(3):
        brain_vallen[k].append(brain_vals[i][k].shape[0] + brain_count[k])
        brain_count[k] += brain_vals[i][k].shape[0]
for i,fb in enumerate(roi_data):
    _, roi_faces, roi_vertices, roi_normals = obtain_data(roi_data[i], roi_label[i]) # 'ans' or 'auditory roi'
    roi_vals.append([roi_faces,roi_vertices,roi_normals])
    for k in range(3):
        roi_vallen[k].append(roi_vals[i][k].shape[0] + roi_count[k])
        roi_count[k] += roi_vals[i][k].shape[0]

# Initialize final variables
brain_faces = np.empty((brain_vallen[0][-1],3))
brain_vertices = np.empty((brain_vallen[1][-1],3))
brain_normals = np.empty((brain_vallen[2][-1],3))
roi_faces = np.empty((roi_vallen[0][-1],3))
roi_vertices = np.empty((roi_vallen[1][-1],3))
roi_normals = np.empty((roi_vallen[2][-1],3))
roi_vertices_weights = np.zeros((roi_vallen[1][-1],)) # matching index weights

# Merge file variables
for i in range(len(brain_data)): # i for each file index
    if i == 0: # first file starts from zero in lists
        brain_faces[0:brain_vallen[0][i]] = brain_vals[i][0]
        brain_vertices[0:brain_vallen[1][i]] = brain_vals[i][1]
        brain_normals[0:brain_vallen[2][i]] = brain_vals[i][2]
    else:
        brain_faces[brain_vallen[0][i-1]:brain_vallen[0][i]] = brain_vals[i][0]
        brain_vertices[brain_vallen[1][i-1]:brain_vallen[1][i]] = brain_vals[i][1]
        brain_normals[brain_vallen[2][i-1]:brain_vallen[2][i]] = brain_vals[i][2]
for i in range(len(roi_data)): # i for each roi file
    if i == 0: # first file starts from zero in lists
        roi_faces[0:roi_vallen[0][i]] = roi_vals[i][0]
        roi_vertices[0:roi_vallen[1][i]] = roi_vals[i][1]
        roi_normals[0:roi_vallen[2][i]] = roi_vals[i][2]

        roi_vertices_weights[0:roi_vallen[1][i]] = np.repeat(roi_weights[i],roi_vallen[1][i])
    else:
        roi_faces[roi_vallen[0][i-1]:roi_vallen[0][i]] = roi_vals[i][0]
        roi_vertices[roi_vallen[1][i-1]:roi_vallen[1][i]] = roi_vals[i][1]
        roi_normals[roi_vallen[2][i-1]:roi_vallen[2][i]] = roi_vals[i][2]
        roi_vertices_weights[roi_vallen[1][i-1]:roi_vallen[1][i]] += np.repeat(roi_weights[i],(roi_vallen[1][i] - roi_vallen[1][i-1]))

# Recenter total ROI based on total brain
recentered_roi = recenter(roi_vertices, brain_vertices)

In [ ]:
### Calculate voltages for best position set ###
voltages = []  # Must be flexible list for different sensor counts
infocap = [] # Collect IC values
sensor_count = fields.shape[-1]

# Manual trajectory injection
manual = False

# Manual trajectory demo
manual_trajectories = [ # Each inside list is (N*6,) for coordinates for N devices
    np.array([0, 0, 5, 0, 0, 180]),
    np.array([0, 3, 4.5, 0, 0, 180,
              0, -3, 6.5, 0, 0, 180]),
    np.array([-.5, -3, 6, 0, 0, 180,
              -.5, 3, 4.5, 0, 0, 180,
              2, 0, 5, 0, 0, 180])
]
""" 
# ACC/ DL-PFC; R/L inner cortex
manual_trajectories = [ # Each inside list is (N*6,) for coordinates for N devices
    np.array([-3, 25, 18, 0, 0, 180]),
    np.array([-5, 27, 18, 0, 0, 180,
             5, 27, 18, 0, 0, 180]),
    np.array([-10, 28, 18, 0, 0, 180,
             7, 28, 18, 0, 0, 180,
             -3, 23, 18, 0, 0, 180]),
    np.array([-5, 24, 18, 0, 0, 180,
              5, 24, 18, 0, 0, 180,
              -13, 27, 18, 0, 0, 180,
              10, 27, 18, 0, 0, 180]),
    np.array([-8, 28, 18, 0, 0, 180,
              5, 28, 18, 0, 0, 180,
              -3, 23, 18, 0, 0, 180,
              -16, 24, 19, 0, 0, 180,
              13, 24, 19, 0, 0, 180])
]
"""

if manual: # Rebuild device count list
    N_file = []
    for t in manual_trajectories:
        N_file.append(t.shape[0]//6)
        if t.shape[0]%6 != 0:
            print("Error: Manual trajectory incorrect.")
            break

# Main loop
for i,N in enumerate(N_file): # each i file of N devices
    if manual:
        voltages.append(find_snr(manual_trajectories[i],-1))
        infocap.append(find_snr(manual_trajectories[i],2))
    else:
        voltages.append(find_snr(best[i],-1))
        infocap.append(find_snr(best[i],2))
    voltages[i] = voltages[i].reshape(-1,N*sensor_count)
    print(f"Done with {N} devices with voltage array shape: {voltages[i].shape}")

### Assess voltage and IC
Not used in paper

In [ ]:
# Sanity check
print(infocap[0].shape)
print(np.count_nonzero(np.isnan(infocap[0])))
print(np.count_nonzero(np.isnan(np.sum(voltages[0],axis=1))))
print(np.where(~np.isnan(infocap[0]))[0])

In [ ]:
### Assess voltages ###
print("Voltage summary")
for i in range(len(N_file)):
    print("N:",N_file[i])
    print("Mean:",np.nanmean(voltages[i]))
    #print("Max:",np.nanmax(voltages[i]))
    #print("Min:",np.nanmin(voltages[i]))
    print("STD:",np.nanstd(voltages[i]))

In [ ]:
### Assess Information Capacity ###
IC_threshold = 160 # Bits/s (X = 100log2(SNR+1))
print("Information capacity summary")
for i in range(len(N_file)):
    print("N:",N_file[i])
    print("Mean:",np.nanmean(np.nanmax(np.nan_to_num(infocap[i]),axis=1)))
    #print("Max:",np.nanmax(np.nanmax(infocap[i],axis=1)))
    #print("Min:",np.nanmin(np.nanmax(infocap[i],axis=1)))
    print("STD:",np.nanstd(np.nanmax(np.nan_to_num(infocap[i]),axis=1)))
    print("Total IC:",np.nansum(np.nanmax(np.nan_to_num(infocap[i]),axis=1)),"Bits/s.")
    print("Percent of sources above threshold:",np.count_nonzero(np.nanmax(np.nan_to_num(infocap[i]),axis=1) > IC_threshold)/infocap[i].shape[0]*100,
          "%",f" for a threshold of {IC_threshold} Bits/s.")

In [ ]:
### Plot IC distribution for one device by distance of sources ###
# Avoid NaN indices that are not observed by the dataset
notnan = np.where(~np.isnan(infocap[0]))[0] # safe indices

# Distance to each point from the device assigned position; matched indices to voltage
distance = recentered_roi - best[0][0:3]
distance = np.sqrt(np.sum(distance**2,axis=-1))[notnan]

# Plot source distances
plt.hist(distance,bins=50, color='lightblue',edgecolor='black')
plt.xlim([0,30])
plt.xlabel('Device-Source Distance (mm)')
plt.ylabel('Source Count')
plt.savefig(path.join(out,f"8_Distance-distribution.png"),dpi=300)
plt.show()

In [ ]:
# Plot IC voilin for DiSc and vSEEG at varying range
bins = [0,7.5,15,22.5,30] # mm distance groups
binned_ic = [[] for _ in range(len(bins)-1)]
positions = []
for i in (range(len(bins)-1)):
    positions.append((bins[i]+bins[i+1])/2)
positions = np.asarray(positions)

for dist, ic in zip(distance,infocap[0][notnan]):
    bin_idx = np.digitize(dist,bins) - 1
    if 0<=bin_idx<(len(bins)-1):
        binned_ic[bin_idx].append(ic[0])

plt.violinplot(binned_ic,positions=positions,showmeans=True,widths=(bins[1]-bins[0])/1.5)
plt.xticks(bins)
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.xlabel('Binned Distance (mm)')
plt.ylabel('Info. Cap. Distribution')
plt.savefig(path.join(out,f"8_IC-distribution-{voltages[0].shape[-1]}e.png"),dpi=300)
plt.show()

### SEPIO

In [ ]:
### SEPIO train/test with variable replicates ###
MC_cycles = 1 # How many Monte-Carlo cycles to complete and average
rep_per_max = 1 # (INT) replicates per max sensors; 1 for DiSc; 4 for SEEG; 8 for ECoG
sensor_div = 16 # Sensor testing granularity (defines sensor_range steps)
noise_mult = 10. # Boost noise uniformly; 5-10 for increased classification difficulty
totalcoefs = [] # Coefficients per test set
totalacc = [] # Test acc (sensor space)
spatialacc = [] # Test acc (source space)
sensor_list = [] # list of sensor_range

for i, N in enumerate(N_file):
    print(f"Starting loop {i+1} with {N} devices.")
    MCcoefs, MCaccs, MCSaccs, sensor_range = sepio.mc_train(X=voltages[i],y=None,Xt=None,yt=None,sensor_div=sensor_div,
    MCcount=MC_cycles,noise=noise*noise_mult,replicates=rep_per_max*voltages[i].shape[-1],nbasis=20,spatial=False,l1=0.001,rep_subdiv=8)
    totalcoefs.append(MCcoefs)
    totalacc.append(MCaccs)
    spatialacc.append(MCSaccs)
    sensor_list.append(sensor_range)

In [ ]:
# Check top-N sensors alone
if False:
    topN = 16
    dataset = 0 # index for file
    rep_per_max = 1
    sensor_div = 2 # Sensor testing granularity (defines sensor_range steps)
    sensors = np.flip(np.argsort(np.mean(np.abs(totalcoefs[dataset]),axis=1)))[:16]
    TNcoefs, TNaccs, TNSaccs, TNsensor_range = sepio.mc_train(X=voltages[dataset][:,sensors],y=None,Xt=None,yt=None,sensor_div=sensor_div,
    MCcount=1,noise=4.1,replicates=rep_per_max*voltages[dataset][:,sensors].shape[-1],nbasis=20,spatial=False,l1=0.001,rep_subdiv=8)

In [ ]:
### Plotting accuracies for each device set ###
# Produces line plots of SEPIO classification accuracy (%) versus sensor count.
# Logic:
# - totalacc[i] holds accuracy progression for device configuration i across tested sensor subsets.
# - For multi-device cases, sensor axis is total sensors (device_count * sensors_per_device).
# - Sensor counts derived from range(0, sensor_count*N, sensor_div) shifted by +sensor_div to start at first non-zero group.
# - Y-axis scaled to percent; X-axis increments enforce granularity defined by sensor_div.
for i,N in enumerate(N_file):
    if i < len(totalacc):
        if N == 1:
            plt.plot(np.arange(0,sensor_count,sensor_div)+sensor_div,100*totalacc[i],label="1 Device")
        else:
            plt.plot(np.arange(0,sensor_count*N,sensor_div)+sensor_div,100*totalacc[i],label=f"{N} Devices")

plt.ylabel(f"% Accuracy for {voltages[0].shape[0]} classes")
plt.xlabel("Sensor count")
plt.title(f"SEPIO classification accuracy\n({data[0]['type'][0]})")
plt.legend(loc='lower right')
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.savefig(path.join(out,f"8_SEPIO_total-{data[0]['type'][0]}.png"),dpi=300)
plt.show()

In [ ]:
### Plot sensorXdevice accuracy contour ###
# Builds a triangulated contour over (sensor_count, device_count) -> accuracy space.
# Data preparation:
# - xs: sensor counts (multiples of sensor_div)
# - ys: device counts (from N_file)
# - zs: accuracy values (percent) from totalacc arrays
# Visualization:
# - `plot_divs` controls contour granularity.
# - Colorbar labels accuracy distribution; can reveal diminishing returns when scaling devices vs sensors.

# Assign plot values
xs = []
ys = []
zs = []
for n in range(len(totalacc)): # k is acc
    # n+1 for each device count
    x = []
    y = []
    for s in range(totalacc[n].size):
        # (s+1)*16 for each sensor count
        xs.append((s+1)*sensor_div)
        ys.append(N_file[n])
        zs.append(totalacc[n][s]*100)

# Plot settings
plot_divs = 18 # number of plot divisions
#plt.tricontour(xs,ys,zs,levels=plot_divs,colors='k') # Adds lines between regions
pt = plt.tricontourf(xs,ys,zs,levels=plot_divs,cmap="RdBu_r") # colors in regions

plt.yticks(N_file[:len(totalacc)])
plt.xlim([0,max(N_file[:len(totalacc)])*fields.shape[-1]])
plt.ylabel("Device count")
plt.xlabel("Sensor count")
plt.title(f"SEPIO by device and sensor count\n({data[0]['type'][0]})")
cb = plt.colorbar(pt)
cb.set_label("% Accuracy")
plt.savefig(path.join(out,f"8_SEPIO_SxN-{data[0]['type'][0]}.png"),dpi=300)
plt.show()